## Import

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
import random
import os

# imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# model learning
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.utils.class_weight import compute_sample_weight
import statsmodels.api as sm

# 평가 지표
from sklearn.metrics import mean_absolute_error

# 모델 저장
import pickle

## Fixed Random-Seed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data

In [3]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
building_info = pd.read_csv('../data/building_info.csv')

## Change Names

In [4]:
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building_info['건물유형'] = building_info['건물유형'].replace(translation_dict)
# building_info.drop('Unnamed: 0', axis = 1 , inplace=True)

In [5]:
train_df.columns = ['num_date_time', '건물번호', '일시', '기온', '강수량', '풍속', '습도',
       '일조', '일사', '전력소비량']

In [6]:
test_df.columns = ['num_date_time', '건물번호', '일시', '기온', '강수량', '풍속', '습도']

In [7]:
building_info.columns = ['건물번호', '건물유형', '연면적', '냉방면적', '태양광용량', 'ESS저장용량',
       'PCS용량']

## Encoding

In [8]:
# one hot encoding 생성
building_info = pd.get_dummies(building_info, columns=['건물유형'], drop_first=True)

In [9]:
# LabelEncoder를 객체로 생성
#encoder = LabelEncoder()

# fit, transform 메소드를 통한 레이블 인코딩
#encoder.fit(building_info['건물유형'])
#building_info['건물유형'] = encoder.transform(building_info['건물유형'])

## Merge building info

In [10]:
building_info['태양광용량'][building_info['태양광용량'] == '-'] = np.nan
building_info['ESS저장용량'][building_info['ESS저장용량'] == '-'] = np.nan
building_info['PCS용량'][building_info['PCS용량'] == '-'] = np.nan

In [11]:
# 태양광 사용하는지 마는지 여부
building_info['태양광사용여부'] = 0

In [12]:
building_info['태양광사용여부'][(building_info['태양광용량'].notnull()) | (building_info['ESS저장용량'].notnull()) | (building_info['PCS용량'].notnull())] = 1

In [13]:
train_df = pd.merge(train_df, building_info, on='건물번호', how='left')
test_df = pd.merge(test_df, building_info, on='건물번호', how='left')

## Train Data Pre-Processing

In [14]:
train_df.isna().sum()

num_date_time                            0
건물번호                                     0
일시                                       0
기온                                       0
강수량                                 160069
풍속                                      19
습도                                       9
일조                                   75182
일사                                   87913
전력소비량                                    0
연면적                                      0
냉방면적                                     0
태양광용량                               130560
ESS저장용량                             193800
PCS용량                               193800
건물유형_Commercial                          0
건물유형_Data Center                         0
건물유형_Department Store and Outlet         0
건물유형_Discount Mart                       0
건물유형_Hospital                            0
건물유형_Hotel and Resort                    0
건물유형_Knowledge Industry Center           0
건물유형_Other Buildings                     0
건물유형_Public

- 결측치가 시급하다. 아무리봐도 처리 해야할 것 같다.

In [15]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['월'] = train_df['일시'].apply(lambda x : int(x[4:6]))
#train_df['일'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['시'] = train_df['일시'].apply(lambda x : int(x[9:11]))

In [16]:
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')

In [17]:
# 요일 추가 (0이 월요일 6이 일요일)
train_df['주'] = train_df['일시'].dt.dayofweek

In [18]:
train_df['일조'][(train_df['일조'].isna()) & ((train_df['시'] <= 5) | (train_df['시'] >= 21))] = 0
train_df['일사'][(train_df['일사'].isna()) & ((train_df['시'] <= 5) | (train_df['시'] >= 21))] = 0

In [19]:
train_df['풍속'][train_df['풍속'].isna()] = 0
train_df['습도'][train_df['습도'].isna()] = 0

In [20]:
# imputation for mice
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(train_df[['기온', '풍속', '습도', '일조', '일사']])

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
train_df[['기온', '풍속', '습도', '일조', '일사']] = pd.DataFrame(imputer_mice.transform(train_df[['기온', '풍속', '습도', '일조', '일사']]),
                         columns=['기온', '풍속', '습도', '일조', '일사'])

In [21]:
# 숫자 보정
train_df['일조'][train_df['일조'] < 0] = 0
train_df['일사'][train_df['일사'] < 0] = 0

In [22]:
# 변수 정리
train_df2 = train_df.drop(columns=['num_date_time', '일시',
                                 '태양광용량','ESS저장용량','PCS용량', '강수량'])

In [23]:
# 강수량 결측치는 0으로 보정 -> 비 안왔을것이라 예상하고 진행함
# train_df2['강수량(mm)'][train_df2['강수량(mm)'].isna()] = 0

In [24]:
# imputation for mice
#imputer_mice = IterativeImputer(random_state=42)
#imputer_mice.fit(train_df2)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
#train_df2 = pd.DataFrame(imputer_mice.transform(train_df2),
#                         columns=train_df2.columns)

In [25]:
# 그냥 0으로 채우기
#train_df2[train_df2.isna()] = 0

## Append Value

In [26]:
# 온도에 따른 포화 수증기압(mb)
#train_df2['포화수증기압'] = 6.11*10**((7.5*train_df2['기온'])/(237.3 + train_df2['기온']))

In [27]:
# 현재 대기의 수증기압(mb)
#train_df2['대기압'] = train_df2['습도'] * train_df2['포화수증기압'] / 100

In [28]:
# 절대습도(g/m^3)
#train_df2['절대습도'] = (0.794*train_df2['대기압'])/(1+0.00366*train_df2['기온'])

In [29]:
# 공기중의 혼합비(mb)
#train_df2['혼합비'] = 0.622 * train_df2['대기압']/(1013.25 - train_df2['대기압'])

In [30]:
# 습구온도
#train_df2['습구온도'] = train_df2['기온']*np.arctan(0.151977+(train_df2['습도'] + 8.313659)**0.5) + np.arctan(train_df2['기온'] + train_df2['습도']) - np.arctan(train_df2['습도'] - 1.676331) +0.00391838*(train_df2['습도']**1.5)*np.arctan(0.023101*train_df2['습도'])-4.686035

In [31]:
# 불쾌지수
#train_df2['불쾌지수'] = 9/5*train_df2['기온'] - 0.55*(1-0.01*train_df2['습도'])*(9/5*train_df2['기온'] - 26) + 32

In [32]:
# 더위체감지수
#train_df2['더위체감지수'] = -0.24418 + 0.553991*train_df2['습구온도'] + 0.455346*train_df2['기온'] - 0.00217*train_df2['습구온도']**2 + 0.002782*train_df2['습구온도']*train_df2['기온'] + 3

In [33]:
# np.select
#condlist = [
#    (train_df2['더위체감지수'] < 21),
#    (train_df2['더위체감지수'] < 25) & (train_df2['더위체감지수'] >= 21),
#    (train_df2['더위체감지수'] < 28) & (train_df2['더위체감지수'] >= 25),
#    (train_df2['더위체감지수'] < 31) & (train_df2['더위체감지수'] >= 28),
#    (train_df2['더위체감지수'] >= 31)
#            ]

#choicelist = [0, 1, 2, 3, 4]

In [34]:
# 범주형 체감지수
#train_df2['더위체감지수_범주'] = np.select(condlist, choicelist)

In [35]:
# 열지수
#train_df2['열지수'] = (5/9)*(-42.379 + (2.04901523 * ((9/5)*train_df2['기온']+32)) + (10.14333127 * train_df2['습도']) - (0.22475541 * ((9/5)*train_df2['기온']+32)*train_df2['습도']) - (6.83783e-3 * ((9/5)*train_df2['기온']+32)**2) - (5.481717e-2 * train_df2['습도']**2) + (1.22874e-3 * ((9/5)*train_df2['기온']+32)**2*train_df2['습도']) + (8.5282e-4 * ((9/5)*train_df2['기온']+32)*train_df2['습도']**2) - (1.99e-6 * ((9/5)*train_df2['기온']+32)**2*train_df2['습도']**2) - 32)

In [36]:
# 평균냉방면적
train_df2['평균냉방면적'] = train_df2['냉방면적']/train_df2['연면적']

In [37]:
# onehot 주
#train_df2 = pd.get_dummies(train_df2, columns=['주'], drop_first=True)

In [38]:
# 훈련 및 테스트 데이터 셋
train_x = train_df2[train_df2.columns.difference(['전력소비량'])]
train_y = train_df2['전력소비량']

# BoxCox

In [39]:
# 가중치
boxcox = 0.21

In [40]:
# 적용
train_y_root_twice = train_y**boxcox

## Value Check

In [41]:
results = sm.OLS(train_y_root_twice, sm.add_constant(train_x)).fit()

In [42]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  전력소비량   R-squared:                       0.400
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     5674.
Date:                Fri, 11 Aug 2023   Prob (F-statistic):               0.00
Time:                        21:16:41   Log-Likelihood:            -1.9070e+05
No. Observations:              204000   AIC:                         3.814e+05
Df Residuals:                  203975   BIC:                         3.817e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                5.7199      0.038    151.673      0.000       5.646       5.794
건물번호                                -0.0475      0.001    -92.041      0.000      -0.048      -0.046
건물유형_Commercial                      0.3288      0.008     39.211      0.000       0.312       0.345
건물유형_Data Center                     0.1810      0.018     10.244      0.000       0.146       0.216
건물유형_Department Store and Outlet    -0.5982      0.014    -41.948      0.000      -0.626      -0.570
건물유형_Discount Mart                   1.4269      0.014     99.534      0.000       1.399       1.455
건물유형_Hospital                        0.2045      0.011     18.318      0.000       0.183       0.226
건물유형_Hotel and Resort                1.7965      0.018    100.120      0.000       1.761       1.832
건물유형_Knowledge Industry Center       1.2686      0.011    118.137      0.000       1.248       1.290
건물유형_Other Buildings                -2.0946      0.030    -70.222      0.000      -2.153      -2.036
건물유형_Public                         -1.7424      0.024    -71.306      0.000      -1.790      -1.694
건물유형_Research Institute              1.2754      0.009    144.122      0.000       1.258       1.293
건물유형_University                     -0.5289      0.021    -25.725      0.000      -0.569      -0.489
기온                                   0.0152      0.001     28.204      0.000       0.014       0.016
냉방면적                             -2.453e-06   4.34e-08    -56.477      0.000   -2.54e-06   -2.37e-06
습도                                   0.0010      0.000      7.406      0.000       0.001       0.001
시                                    0.0134      0.000     63.108      0.000       0.013       0.014
연면적                               2.102e-06   3.69e-08     57.022      0.000    2.03e-06    2.17e-06
월                                    0.0572      0.002     25.405      0.000       0.053       0.062
일사                                   0.2093      0.003     71.295      0.000       0.204       0.215
일조                                  -0.2016      0.007    -27.949      0.000      -0.216      -0.187
주                                   -0.0183      0.001    -26.328      0.000      -0.020      -0.017
태양광사용여부                             -0.0067      0.004     -1.774      0.076      -0.014       0.001
평균냉방면적                               0.5464      0.009     62.836      0.000       0.529       0.563
풍속                                   0.0625      0.001     57.425      0.000       0.060       0.065
==============================================================================
Omnibus:                     7835.515   Durbin-Watson:                   0.070
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10150.233
Skew:             

## Standard Scaling

In [43]:
# 표준화
#std_scaler = StandardScaler()
#train_x = std_scaler.fit_transform(train_x)

## Regression Model Fit

In [44]:
# automl modeling
automl = AutoML(mode="Compete",
                total_time_limit = 3600*3,
                algorithms=["Baseline",
                            "CatBoost",
                            "Xgboost",
                            "Random Forest",
                            "Extra Trees",
                            "LightGBM",
                            "Neural Network"], 
                ml_task = "regression",
                eval_metric = 'mae',
                random_state = 42,
                results_path = '../model/AutoML_OneHot_0.21_imputate_2value_sunbattery_NA0/'
               )

In [45]:
# fitting
automl.fit(train_x, train_y_root_twice)

AutoML directory: ../model/AutoML_OneHot_0.21_imputate_2value_sunbattery_NA0/
The task is regression with evaluation metric mae
AutoML will use algorithms: ['Baseline', 'CatBoost', 'Xgboost', 'Random Forest', 'Extra Trees', 'LightGBM', 'Neural Network']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 0.426946 trained in 1.02 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 1 model
1_Baseline mae 0.620316 trained in 4.04 seconds
* Step default_algorithms will try to check up to 6 models
2_Default_LightGBM mae 0.045742 trained in 1075.5

AutoML(algorithms=['Baseline', 'CatBoost', 'Xgboost', 'Random Forest',
                   'Extra Trees', 'LightGBM', 'Neural Network'],
       eval_metric='mae', ml_task='regression', mode='Compete', random_state=42,
       results_path='../model/AutoML_OneHot_0.21_imputate_2value_sunbattery_NA0/',
       total_time_limit=10800)

## Test Data Pre-Processing

In [46]:
test_df['월'] = test_df['일시'].apply(lambda x : int(x[4:6]))
#test_df['일'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['시'] = test_df['일시'].apply(lambda x : int(x[9:11]))

In [47]:
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')

In [48]:
# 요일 추가 (0이 월요일 6이 일요일)
test_df['주'] = test_df['일시'].dt.dayofweek

In [49]:
# 없는 변수 추가하기
test_df['일조'] = np.nan
test_df['일사'] = np.nan

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
test_df[['기온', '풍속', '습도', '일조', '일사']] = pd.DataFrame(imputer_mice.transform(test_df[['기온', '풍속', '습도', '일조', '일사']]),
                         columns=['기온', '풍속', '습도', '일조', '일사'])

In [50]:
# 숫자 보정
test_df['일조'][test_df['일조'] < 0] = 0
test_df['일사'][test_df['일사'] < 0] = 0

In [51]:
# 변수 정리
test_df2 = test_df.drop(columns=['num_date_time', '일시',
                                 '태양광용량','ESS저장용량','PCS용량', '강수량'])

## Append Value

In [52]:
# 온도에 따른 포화 수증기압(mb)
#test_df2['포화수증기압'] = 6.11*10**((7.5*test_df2['기온'])/(237.3 + test_df2['기온']))

In [53]:
# 현재 대기의 수증기압(mb)
#test_df2['대기압'] = test_df2['습도'] * test_df2['포화수증기압'] / 100

In [54]:
# 절대습도(g/m^3)
#test_df2['절대습도'] = (0.794*test_df2['대기압'])/(1+0.00366*test_df2['기온'])

In [55]:
# 공기중의 혼합비(mb)
#test_df2['혼합비'] = 0.622 * test_df2['대기압']/(1013.25 - test_df2['대기압'])

In [56]:
# 습구온도
#test_df2['습구온도'] = test_df2['기온']*np.arctan(0.151977+(test_df2['습도'] + 8.313659)**0.5) + np.arctan(test_df2['기온'] + test_df2['습도']) - np.arctan(test_df2['습도'] - 1.676331) +0.00391838*(test_df2['습도']**1.5)*np.arctan(0.023101*test_df2['습도'])-4.686035

In [57]:
# 불쾌지수
#test_df2['불쾌지수'] = 9/5*test_df2['기온'] - 0.55*(1-0.01*test_df2['습도'])*(9/5*test_df2['기온'] - 26) + 32

In [58]:
# 더위체감지수
#test_df2['더위체감지수'] = -0.24418 + 0.553991*test_df2['습구온도'] + 0.455346*test_df2['기온'] - 0.00217*test_df2['습구온도']**2 + 0.002782*test_df2['습구온도']*test_df2['기온'] + 3

In [59]:
# np.select
#condlist = [
#    (test_df2['더위체감지수'] < 21),
#    (test_df2['더위체감지수'] < 25) & (test_df2['더위체감지수'] >= 21),
#    (test_df2['더위체감지수'] < 28) & (test_df2['더위체감지수'] >= 25),
#    (test_df2['더위체감지수'] < 31) & (test_df2['더위체감지수'] >= 28),
#    (test_df2['더위체감지수'] >= 31)
#           ]

#choicelist = [0, 1, 2, 3, 4]

In [60]:
# 범주형 체감지수
#test_df2['더위체감지수_범주'] = np.select(condlist, choicelist)

In [61]:
# 열지수
#test_df2['열지수'] = (5/9)*(-42.379 + (2.04901523 * ((9/5)*test_df2['기온']+32)) + (10.14333127 * test_df2['습도']) - (0.22475541 * ((9/5)*test_df2['기온']+32)*test_df2['습도']) - (6.83783e-3 * ((9/5)*test_df2['기온']+32)**2) - (5.481717e-2 * test_df2['습도']**2) + (1.22874e-3 * ((9/5)*test_df2['기온']+32)**2*test_df2['습도']) + (8.5282e-4 * ((9/5)*test_df2['기온']+32)*test_df2['습도']**2) - (1.99e-6 * ((9/5)*test_df2['기온']+32)**2*test_df2['습도']**2) - 32)

In [62]:
# 평균냉방면적
test_df2['평균냉방면적'] = test_df2['냉방면적']/test_df2['연면적']

In [63]:
# onehot 주
#test_df2 = pd.get_dummies(test_df2, columns=['주'], drop_first=True)

In [64]:
# test_df2 = std_scaler.fit_transform(test_df2)

In [65]:
# 훈련 및 테스트 데이터 셋
#test_df2 = test_df2[test_df2.columns.difference(['냉방면적', '연면적', '습도', '월', '건물번호', '기온'])]

## Inference

In [66]:
preds = automl.predict(test_df2)

## Submission

In [67]:
submission = pd.read_csv('../data/sample_submission.csv')
submission

,num_date_time,answer
0,1_20220825 00,0
1,1_20220825 01,0
2,1_20220825 02,0
3,1_20220825 03,0
4,1_20220825 04,0
...,...,...
16795,100_20220831 19,0
16796,100_20220831 20,0
16797,100_20220831 21,0
16798,100_20220831 22,0


In [68]:
submission['answer'] = preds**(1/boxcox)
submission

,num_date_time,answer
0,1_20220825 00,2277.779088
1,1_20220825 01,2129.518909
2,1_20220825 02,2094.568663
3,1_20220825 03,1936.096423
4,1_20220825 04,1935.976379
...,...,...
16795,100_20220831 19,892.203778
16796,100_20220831 20,786.494305
16797,100_20220831 21,700.185735
16798,100_20220831 22,608.916946


In [69]:
submission.to_csv('../data/AutoML_OneHot_0.21_imputate_2value_sunbattery_NA0.csv', index=False)